In [1]:
import numpy as np
import pickle as pickle 

In [2]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [3]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we
                            #can use it as a key in a dict   

In [4]:
data = loadDataSet()
c1 = createC1(data)
c1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [5]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt:
                    ssCnt[can]=1
                else: 
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.append(key) # insert(0,key) #
        supportData[key] = support
    return retList, supportData

In [6]:
retList, supportData = scanD(data, c1, 0.5)

In [7]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            # 如果只有 {1, 3},{2, 3}没有出现{1,2} 那{1,2,3}一定不符合，
            # 所以 就不需要计算{1,2,3}的结果
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

In [8]:
l2k = aprioriGen(retList,2)

In [9]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = dataSet
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 1):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        if len(Lk)>0 :
            L.append(Lk)
        else:
            break
        k += 1
    return L, supportData

In [10]:
apriori(data, 0.5)

([[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
  [frozenset({1, 3}), frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5})],
  [frozenset({2, 3, 5})]],
 {frozenset({1}): 0.5,
  frozenset({3}): 0.75,
  frozenset({4}): 0.25,
  frozenset({2}): 0.75,
  frozenset({5}): 0.75,
  frozenset({1, 3}): 0.5,
  frozenset({2, 3}): 0.5,
  frozenset({3, 5}): 0.5,
  frozenset({2, 5}): 0.75,
  frozenset({1, 2}): 0.25,
  frozenset({1, 5}): 0.25,
  frozenset({2, 3, 5}): 0.5})

In [11]:
apriori(data, 0.7)

([[frozenset({3}), frozenset({2}), frozenset({5})], [frozenset({2, 5})]],
 {frozenset({1}): 0.5,
  frozenset({3}): 0.75,
  frozenset({4}): 0.25,
  frozenset({2}): 0.75,
  frozenset({5}): 0.75,
  frozenset({2, 3}): 0.5,
  frozenset({3, 5}): 0.5,
  frozenset({2, 5}): 0.75})

In [12]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            #print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [13]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [14]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else: # i=1 ， freqSet 的长度为 2 ，只能有一个规制
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList   

In [15]:
def printRules(ruleList, itemMeaning):
    for ruleTup in ruleList:
        for item in ruleTup[0]:
            print(itemMeaning[item])
        print ("           -------->")
        for item in ruleTup[1]:
            print (itemMeaning[item])
        print ("confidence: %f" % ruleTup[2])
        print  

In [16]:
L,supportData = apriori(data, 0.5)
rules = generateRules(L,supportData,minConf=0.5)
rules

[(frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

In [17]:
def loadBills(fileName):      #general function to parse tab -delimited floats
    dataMat = []                #assume last column is target value
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        bill = {}
        bill['id'] = curLine[0]
        bill['title'] = curLine[1]
        dataMat.append(bill)
    return dataMat

In [18]:
bills20 = loadBills('recent20bills.txt')
#bills100 = loadBills('recent100bills.txt')
f = open('meaning20.txt', 'rb')
items = pickle.load(f)
f.close()
#items

In [19]:
f = open('bills20DataSet.txt', 'rb')
data = pickle.load(f)
f.close()
#data

In [20]:
L, suppData = apriori(data, 0.3)
len(L)

7

In [21]:
rules = generateRules(L,suppData,minConf=0.95)

In [22]:
#rules[0][0]
for r in rules:
    print(items[list(r[0])[0]]) 
    print('---------->', items[list(r[1])[0]], '\t%.2f%%:'%(r[2]*100))

Repealing the Health Care Bill -- Yea
----------> Republican 	98.37%:
Prohibiting Use of Federal Funds For Planned Parenthood -- Yea
----------> Republican 	95.42%:
Repealing the Health Care Bill -- Yea
----------> Terminating the Home Affordable Modification Program -- Yea 	96.33%:
Prohibiting Use of Federal Funds For Planned Parenthood -- Yea
----------> Terminating the Home Affordable Modification Program -- Yea 	95.00%:
Prohibiting Use of Federal Funds For Planned Parenthood -- Yea
----------> Repealing the Health Care Bill -- Yea 	97.08%:
Repealing the Health Care Bill -- Yea
----------> Prohibiting Use of Federal Funds For Planned Parenthood -- Yea 	95.10%:
Repealing "Don't Ask, Don't Tell" After Military Review and Certification -- Nay
----------> Prohibiting 2010- 2011 Congressional Cost-of-Living Pay Increase -- Yea 	95.36%:
Mine Safety Act -- Nay
----------> Prohibiting 2010- 2011 Congressional Cost-of-Living Pay Increase -- Yea 	96.34%:
Prohibiting Federal Funding of Nationa

In [23]:
mushDatSet = [line.split() for line in open('mushroom.dat').readlines()]

In [24]:
L,supportData = apriori(mushDatSet,0.3)

In [25]:
#for li in L :
for item in L[2] : #li :
    if item.intersection('2'):#交集
        print(item)

frozenset({'34', '2', '23'})
frozenset({'59', '2', '23'})
frozenset({'63', '2', '23'})
frozenset({'85', '2', '23'})
frozenset({'86', '2', '23'})
frozenset({'93', '2', '23'})
frozenset({'34', '2', '36'})
frozenset({'59', '34', '2'})
frozenset({'34', '63', '2'})
frozenset({'34', '2', '67'})
frozenset({'76', '34', '2'})
frozenset({'34', '85', '2'})
frozenset({'86', '34', '2'})
frozenset({'34', '2', '90'})
frozenset({'34', '2', '93'})
frozenset({'59', '2', '36'})
frozenset({'63', '36', '2'})
frozenset({'85', '36', '2'})
frozenset({'86', '2', '36'})
frozenset({'93', '2', '36'})
frozenset({'59', '63', '2'})
frozenset({'59', '85', '2'})
frozenset({'59', '86', '2'})
frozenset({'59', '2', '90'})
frozenset({'59', '93', '2'})
frozenset({'85', '63', '2'})
frozenset({'86', '63', '2'})
frozenset({'63', '90', '2'})
frozenset({'93', '63', '2'})
frozenset({'85', '67', '2'})
frozenset({'86', '2', '67'})
frozenset({'76', '85', '2'})
frozenset({'76', '86', '2'})
frozenset({'86', '85', '2'})
frozenset({'85